In [2]:
import numpy as np
import cv2


filename = 'mydata.json'
# filename = 'mydata.xml'
# filename = 'mydata.yml'


def writeData():
    name = 'Jane'
    age = 10
    pt1 = (100, 200)
    scores = (80, 90, 50)
    mat1 = np.array([[1.0, 1.5], [2.0, 3.2]], dtype=np.float32)

    fs = cv2.FileStorage(filename, cv2.FILE_STORAGE_WRITE)

    if not fs.isOpened():
        print('File open failed!')
        return

    fs.write('name', name)
    fs.write('age', age)
    fs.write('point', pt1)
    fs.write('scores', scores)
    fs.write('data', mat1)

    fs.release()


def readData():
    fs = cv2.FileStorage(filename, cv2.FILE_STORAGE_READ)

    if not fs.isOpened():
        print('File open failed!')
        return

    name = fs.getNode('name').string()
    age = int(fs.getNode('age').real())
    pt1 = tuple(fs.getNode('point').mat().astype(np.int32).flatten())
    scores = tuple(fs.getNode('scores').mat().flatten())
    mat1 = fs.getNode('data').mat()

    fs.release()

    print('name:', name)
    print('age:', age)
    print('point:', pt1)
    print('scores:', scores)
    print('data:')
    print(mat1)


if __name__ == '__main__':
    writeData()
    readData()


name: Jane
age: 10
point: (100, 200)
scores: (80.0, 90.0, 50.0)
data:
[[1.  1.5]
 [2.  3.2]]


In [3]:
import numpy as np
import cv2


def saturated(value):
    if value > 255:
        value = 255
    elif value < 0:
        value = 0

    return value


def on_level_change(pos):
    img[:] = saturated(pos * 16)
    cv2.imshow('image', img)


img = np.zeros((400, 400), np.uint8)

cv2.namedWindow('image')
cv2.createTrackbar('level', 'image', 0, 16, on_level_change)

cv2.imshow('image', img)
cv2.waitKey()
cv2.destroyAllWindows()


In [1]:
import numpy as np
import cv2


def mask_setTo():
    src = cv2.imread('lenna.bmp', cv2.IMREAD_COLOR)
    mask = cv2.imread('mask_smile.bmp', cv2.IMREAD_GRAYSCALE)

    if src is None or mask is None:
        print('Image load failed!')
        return

    src[mask > 0] = (0, 255, 255)

    cv2.imshow('src', src)
    cv2.imshow('mask', mask)
    cv2.waitKey()
    cv2.destroyAllWindows()


def mask_copyTo():
    src = cv2.imread('airplane.bmp', cv2.IMREAD_COLOR)
    mask = cv2.imread('mask_plane.bmp', cv2.IMREAD_GRAYSCALE)
    dst = cv2.imread('field.bmp', cv2.IMREAD_COLOR)

    if src is None or mask is None or dst is None:
        print('Image load failed!')
        return

    cv2.copyTo(src, mask, dst)
    # dst[mask > 0] = src[mask > 0]

    cv2.imshow('src', src)
    cv2.imshow('dst', dst)
    cv2.imshow('mask', mask)
    cv2.waitKey()
    cv2.destroyAllWindows()


def time_inverse():
    src = cv2.imread('lenna.bmp', cv2.IMREAD_GRAYSCALE)

    if src is None:
        print('Image load failed!')
        return

    dst = np.empty(src.shape, dtype=src.dtype)

    tm = cv2.TickMeter()
    tm.start()

    for y in range(src.shape[0]):
        for x in range(src.shape[1]):
            dst[y, x] = 255 - src[y, x]

    tm.stop()
    print('Image inverse implementation took %4.3f ms.' % tm.getTimeMilli())

    cv2.imshow('src', src)
    cv2.imshow('dst', dst)
    cv2.waitKey()
    cv2.destroyAllWindows()


def useful_func():
    img = cv2.imread('lenna.bmp', cv2.IMREAD_GRAYSCALE)

    if img is None:
        print('Image load failed!')
        return

    sum_img = np.sum(img)
    mean_img = np.mean(img, dtype=np.int32)
    print('Sum:', sum_img)
    print('Mean:', mean_img)

    minVal, maxVal, minPos, maxPos = cv2.minMaxLoc(img)
    print('minVal is', minVal, 'at', minPos)
    print('maxVal is', maxVal, 'at', maxPos)

    src = np.array([[-1, -0.5, 0, 0.5, 1]], dtype=np.float32)
    dst = cv2.normalize(src, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    print('src:', src)
    print('dst:', dst)

    print('round(2.5) is', round(2.5))
    print('round(2.51) is', round(2.51))
    print('round(3.499) is', round(3.499))
    print('round(3.5) is', round(3.5))


if __name__ == '__main__':
    mask_setTo()
    mask_copyTo()
    time_inverse()
    useful_func()

Image inverse implementation took 1294.350 ms.
Sum: 28890183
Mean: 110
minVal is 22.0 at (508, 71)
maxVal is 239.0 at (117, 272)
src: [[-1.  -0.5  0.   0.5  1. ]]
dst: [[  0  64 128 191 255]]
round(2.5) is 2
round(2.51) is 3
round(3.499) is 3
round(3.5) is 4


In [ ]:
import numpy as np
import cv2


def camera_in():
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Camera open failed!")
        return

    print('Frame width:', int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)))
    print('Frame height:', int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

    while True:
        ret, frame = cap.read()

        if not ret:
            break

        inversed = ~frame

        cv2.imshow('frame', frame)
        cv2.imshow('inversed', inversed)

        if cv2.waitKey(10) == 27:
            break

    cv2.destroyAllWindows()


def video_in():
    cap = cv2.VideoCapture('stopwatch.avi')

    if not cap.isOpened():
        print("Video open failed!")
        return

    print('Frame width:', int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)))
    print('Frame height:', int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    print('Frame count:', int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))

    fps = cap.get(cv2.CAP_PROP_FPS)
    print('FPS:', fps)

    delay = round(1000 / fps)

    while True:
        ret, frame = cap.read()

        if not ret:
            break

        inversed = ~frame

        cv2.imshow('frame', frame)
        cv2.imshow('inversed', inversed)

        if cv2.waitKey(delay) == 27:
            break

    cv2.destroyAllWindows()


def camera_in_video_out():
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Camera open failed!")
        return

    w = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    fourcc = cv2.VideoWriter_fourcc(*'DIVX') # *'DIVX' == 'D', 'I', 'V', 'X'
    delay = round(1000 / fps)

    outputVideo = cv2.VideoWriter('output.avi', fourcc, fps, (w, h))

    if not outputVideo.isOpened():
        print('File open failed!')
        return

    while True:
        ret, frame = cap.read()

        if not ret:
            break

        inversed = ~frame

        outputVideo.write(inversed)

        cv2.imshow('frame', frame)
        cv2.imshow('inversed', inversed)

        if cv2.waitKey(delay) == 27:
            break

    cv2.destroyAllWindows()


if __name__ == '__main__':
    camera_in()
    video_in()
    camera_in_video_out()


Frame width: 640
Frame height: 480
Frame width: 640
Frame height: 480
Frame count: 378
FPS: 30.0
